In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import numpy as np
import tensorflow as tf
seed_value = 42
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
random.seed(seed_value)

In [ ]:
embedding_index = {}
import numpy as np
with open('/content/drive/MyDrive/Colab Notebooks/Hsoub/L9/cc.ar.300.vec', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs
print("len(embedding_index)", len(embedding_index))

len(embedding_index) 2000000


In [ ]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Hsoub/L9/data/Arabic Sentiment Analysis Dataset - SS2030.csv',sep=';')

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import string

def cleanText(text):
    numbers="0123456789"
    arabic_punctuation='''`÷×؛<>_()*^ـ،/:"؟.,'~¦+|!”…“–ـ'''
    english_punctuation=string.punctuation
    del_chars=english_punctuation+arabic_punctuation+numbers

    for char in del_chars:
        print()
        text = text.replace(char, "")
    text = text.replace('\n', ' ')
    text = text.strip(' ')
    listStopwords = stopwords.words('arabic')
    tokens_list=word_tokenize(text)
    filtered = []
    for txt in tokens_list:
        if txt not in listStopwords:
            filtered.append(txt)
    return filtered
data['text'] = data['text'].apply(cleanText)
texts = data['text'].values
labels = data['Sentiment'].values


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
from keras.preprocessing.sequence import pad_sequences
maxlen = max([len(seq) for seq in sequences])
sequences_padded = pad_sequences(sequences, maxlen=maxlen)
vocab_size = len(tokenizer.word_index )+1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sequences_padded, labels, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split( X_test, y_test, test_size=0.5, random_state=42)

In [ ]:
embedding_dim = 300
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [ ]:
word_index = tokenizer.word_index
for word,id in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[id] = embedding_vector

In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(Input (shape=(maxlen,)))
embedding_dim = 300
model.add(Embedding(input_dim= vocab_size,
                    input_length=maxlen,
                    output_dim= embedding_dim,
                    weights=[embedding_matrix],
                    trainable=False
                    ))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
epochs=5
batch_size=16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

Epoch 1/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 9s 34ms/step - accuracy: 0.6451 - loss: 0.6380 - val_accuracy: 0.7868 - val_loss: 0.4892
Epoch 2/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.7863 - loss: 0.4705 - val_accuracy: 0.7821 - val_loss: 0.4759
Epoch 3/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.8187 - loss: 0.4093 - val_accuracy: 0.7837 - val_loss: 0.4829
Epoch 4/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - accuracy: 0.8332 - loss: 0.3719 - val_accuracy: 0.7790 - val_loss: 0.4563
Epoch 5/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - accuracy: 0.8429 - loss: 0.3442 - val_accuracy: 0.7727 - val_loss: 0.4733


In [ ]:
y_pred_probs = model.predict(X_test)

y_pred = (y_pred_probs > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = round(100*accuracy_score(y_test, y_pred),2)
print(f'Test Accuracy: {accuracy:.2f}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step
Test Accuracy: 80.72


In [ ]:
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(Input (shape=(maxlen,)))
embedding_dim = 300
model.add(Embedding(input_dim= vocab_size,
                    input_length=maxlen,
                    output_dim= embedding_dim,
                    weights=[embedding_matrix],
                    trainable=True
                    ))
model.add(LSTM(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
epochs=5
batch_size=16
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val))

Epoch 1/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 28s 139ms/step - accuracy: 0.6842 - loss: 0.5690 - val_accuracy: 0.8150 - val_loss: 0.4689
Epoch 2/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 40s 135ms/step - accuracy: 0.9597 - loss: 0.1059 - val_accuracy: 0.8746 - val_loss: 0.4060
Epoch 3/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 41s 137ms/step - accuracy: 0.9985 - loss: 0.0071 - val_accuracy: 0.8589 - val_loss: 0.4533
Epoch 4/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 40s 129ms/step - accuracy: 0.9997 - loss: 0.0016 - val_accuracy: 0.8589 - val_loss: 0.4928
Epoch 5/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 42s 134ms/step - accuracy: 1.0000 - loss: 6.8984e-04 - val_accuracy: 0.8495 - val_loss: 0.5953


In [ ]:
y_pred_probs = model.predict(X_test)

y_pred = (y_pred_probs > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = round(100*accuracy_score(y_test, y_pred),2)
print(f'Test Accuracy: {accuracy:.2f}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
Test Accuracy: 85.42
